# 5-bus Market simulation with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies and Data
First, let's create `System`s to represent the Day-Ahead and Real-Time market clearing
process with hourly, and 5-minute forecast data, respectively.

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))

"/Users/cbarrows/Documents/repos/SIIPExamples.jl"

### Modeling Packages

In [2]:
using InfrastructureSystems
const IS = InfrastructureSystems
using PowerSystems
const PSY = PowerSystems
using PowerSimulations
const PSI = PowerSimulations

PowerSimulations

### Data management packages

In [3]:
using Dates
using DataFrames

### Optimization packages

In [4]:
using JuMP
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### 5-bus Data
The five bus system data here includes hourly day-ahead data, 5-minute real-time market
data, and 6-second actual data.

In [5]:
base_dir = PSY.download(PSY.TestData; branch = "master");
pm_data = PSY.PowerModelsData(joinpath(base_dir, "matpower", "case5_re_uc.m"))

FORECASTS_DIR = joinpath(base_dir,"forecasts","5bus_ts","7day")

tsp_da = IS.read_time_series_metadata(joinpath(FORECASTS_DIR,"timeseries_pointers_da_7day.json"))
tsp_rt = IS.read_time_series_metadata(joinpath(FORECASTS_DIR,"timeseries_pointers_rt_7day.json"))
tsp_agc = IS.read_time_series_metadata(joinpath(FORECASTS_DIR,"timeseries_pointers_agc_7day.json"))

sys_DA = System(pm_data)
reserves = [VariableReserve{ReserveUp}("REG1", 5.0, 0.1),
            VariableReserve{ReserveUp}("REG2", 5.0, 0.06),
            VariableReserve{ReserveUp}("REG3", 5.0, 0.03),
            VariableReserve{ReserveUp}("REG4", 5.0, 0.02)]
contributing_devices = get_components(Generator, sys_DA)
for r in reserves
    add_service!(sys_DA, r, contributing_devices)
end

add_forecasts!(sys_DA, tsp_da)

sys_RT = System(pm_data)
add_forecasts!(sys_RT, tsp_rt)

sys_AGC = System(pm_data)
add_forecasts!(sys_AGC, tsp_agc)

┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/pm_io/matpower.jl:332
┌ Info: extending matpower format with data: gen_name 5x4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/pm_io/matpower.jl:332
┌ Info: extending matpower format by appending matrix "gen_name" in to "gen"
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/pm_io/matpower.jl:664
┌ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/pm_io/data.jl:1218
┌ Info: the voltage setpoint on generator 4 does not match the value at bus 4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/pm_io/data.jl:1686
┌ Info: the voltage setpoint on generator 1 does not match the value at bus 1
└ @ PowerSystems /Users/cbarrows/.julia/packages/Po

## `OperationsProblemTemplate`s

In [6]:
template_uc = template_unit_commitment()
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatch),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, RenewableFixed),
)
template_ed = template_economic_dispatch(devices = devices)


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = HydroDispatch
        formulation = HydroDispatchRunOfRiver
      Generators:
        device_type = ThermalStandard
        formulation = ThermalDispatch
      Ren:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RenFx:
        device_type = RenewableFix
        formulation = RenewableFixed
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
        device_type = HVDCLine
        formulation = HVDCDispatch
  services: 


### Define the Simulation Sequence

In [7]:
stages_definition = Dict("UC" => Stage(GenericOpProblem, template_uc, sys_DA, solver),
                         "ED" => Stage(GenericOpProblem, template_ed, sys_RT, solver))


feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

feedforward = Dict(("ED", :devices, :Generators) => SemiContinuousFF(binary_from_stage = PSI.ON,
                                                         affected_variables = [PSI.ACTIVE_POWER]))

cache = Dict("UC" => [TimeStatusChange(PSY.ThermalStandard, PSI.ON)])

order = Dict(1 => "UC", 2 => "ED")
horizons = Dict("UC" => 24, "ED" =>12)
intervals = Dict("UC" => (Hour(24), Consecutive()),
                 "ED" => (Hour(1), Consecutive()))

DA_RT_sequence = SimulationSequence(step_resolution = Hour(24),
                                    order = order,
                                    horizons = horizons,
                                    intervals = intervals,
                                    ini_cond_chronology = InterStageChronology(),
                                    feedforward_chronologies = feedforward_chronologies,
                                    feedforward = feedforward,
                                    #cache = cache,
                                    )

Feed Forward Chronology
-----------------------

ED: SemiContinuousFF -> Generators

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`

In [8]:
file_path = tempdir()
sim = Simulation(name = "5bus-test",
                steps = 1,
                stages = stages_definition,
                stages_sequence = DA_RT_sequence,
                simulation_folder = file_path)

Simulation()


### Build simulation

In [9]:
build!(sim)

┌ Info: Building Stage 2-ED
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/TTXVL/src/core/simulation.jl:317
┌ Warning: The data doesn't include devices of type HydroDispatch, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/TTXVL/src/devices_models/device_constructors/common/constructor_validations.jl:4
┌ Warning: The data doesn't include devices of type RenewableDispatch, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/TTXVL/src/devices_models/device_constructors/common/constructor_validations.jl:4
┌ Warning: The data doesn't include devices of type RenewableFix, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/TTXVL/src/devices_models/device_constructors/common/constructor_validations.jl:4
┌ Info: Building Stage 1-UC
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/TTXVL/

### Execute simulation

In [10]:
sim_results = execute!(sim)

┌ Info: Serialized time series data to /var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/5bus-test/2020-03-05T08-16-00/models_json/stage_ED_model/ED_sys_data_time_series_storage.h5.
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/qvTbP/src/time_series_storage.jl:47
┌ Info: Serialized System to /var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/5bus-test/2020-03-05T08-16-00/models_json/stage_ED_model/ED_sys_data.json
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/qvTbP/src/serialization.jl:8
┌ Info: Serialized time series data to /var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/5bus-test/2020-03-05T08-16-00/models_json/stage_UC_model/UC_sys_data_time_series_storage.h5.
└ @ InfrastructureSystems /Users/cbarrows/.julia/packages/InfrastructureSystems/qvTbP/src/time_series_storage.jl:47
┌ Info: Serialized System to /var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/5bus-test/2020-03-05T08-16-00/models_json/stage_UC_model/UC_sys_dat

Simulation Results Reference

Results Folder: /var/folders/27/2jr8c7gn4j72fvrg4qt81zrw8w_711/T/5bus-test/2020-03-05T08-16-00/results

Reference Tables

stage-UC

Stop_ThermalStandard: dataframe size (1, 3)

REG3_VariableReserve{ReserveUp}: dataframe size (1, 3)

P_ThermalStandard: dataframe size (1, 3)

REG1_VariableReserve{ReserveUp}: dataframe size (1, 3)

On_ThermalStandard: dataframe size (1, 3)

Start_ThermalStandard: dataframe size (1, 3)

REG2_VariableReserve{ReserveUp}: dataframe size (1, 3)

REG4_VariableReserve{ReserveUp}: dataframe size (1, 3)

stage-ED

P_ThermalStandard: dataframe size (24, 3)

stage-UC

time length: 24

stage-ED

time length: 12

## Results

In [11]:
uc_results = load_simulation_results(sim_results, "UC");
ed_results = load_simulation_results(sim_results, "ED");

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*